In [0]:
import torch
from torch import Tensor
print(torch.__version__)

2.6.0+cu124


In [0]:

# Install required packages.
import os
os.environ['TORCH'] = torch.__version__

In [0]:
#Connect to the Graph Database
import pandas as pd
import numpy as np
from gqlalchemy import Memgraph
from gqlalchemy import match
from gqlalchemy.query_builders.memgraph_query_builder import Operator

# Make a connection to the database
memgraph = Memgraph(host='alzkb.ai', port=7687)

# EDA

In [0]:
labels = memgraph.execute_and_fetch(
    "MATCH (n) UNWIND labels(n) AS lab RETURN DISTINCT lab"
)
nodes = [row["lab"] for row in labels]
print("Node labels:", nodes)


Node labels: ['Drug', 'Gene', 'BiologicalProcess', 'Pathway', 'MolecularFunction', 'CellularComponent', 'Symptom', 'BodyPart', 'DrugClass', 'Disease', 'TranscriptionFactor']


In [0]:
# 1.2 Count nodes per label
counts = memgraph.execute_and_fetch(
    "MATCH (n) UNWIND labels(n) AS lab "
    "RETURN lab, count(*) AS cnt ORDER BY cnt DESC"
)
df_counts = pd.DataFrame(counts)
display(df_counts)


lab,cnt
Gene,193279
Drug,16581
BiologicalProcess,12322
Pathway,4516
MolecularFunction,3460
CellularComponent,1695
BodyPart,652
TranscriptionFactor,519
Symptom,505
DrugClass,474


In [0]:

# 1.3 List all relationship types
rels = memgraph.execute_and_fetch(
    "MATCH ()-[r]->() RETURN DISTINCT type(r) AS rel"
)
print("Rel types:", [row["rel"] for row in rels])

Rel types: ['TRANSCRIPTIONFACTORINTERACTSWITHGENE', 'BODYPARTOVEREXPRESSESGENE', 'BODYPARTUNDEREXPRESSESGENE', 'CHEMICALBINDSGENE', 'CHEMICALINCREASESEXPRESSION', 'CHEMICALDECREASESEXPRESSION', 'GENEREGULATESGENE', 'GENEINTERACTSWITHGENE', 'GENECOVARIESWITHGENE', 'GENEPARTICIPATESINBIOLOGICALPROCESS', 'GENEINPATHWAY', 'GENEHASMOLECULARFUNCTION', 'GENEASSOCIATEDWITHCELLULARCOMPONENT', 'DISEASELOCALIZESTOANATOMY', 'DRUGINCLASS', 'GENEASSOCIATESWITHDISEASE', 'DRUGTREATSDISEASE', 'DRUGCAUSESEFFECT', 'SYMPTOMMANIFESTATIONOFDISEASE']


In [0]:
# 1.4 Count relationships per type
rel_counts = memgraph.execute_and_fetch(
    "MATCH ()-[r]->() RETURN type(r) AS rel, count(*) AS cnt "
    "ORDER BY cnt DESC"
)
df_rel_counts = pd.DataFrame(rel_counts)
display(df_rel_counts)

rel,cnt
GENEPARTICIPATESINBIOLOGICALPROCESS,548285
GENEREGULATESGENE,263978
GENEINPATHWAY,178991
GENEINTERACTSWITHGENE,147088
GENEHASMOLECULARFUNCTION,104752
BODYPARTUNDEREXPRESSESGENE,102185
BODYPARTOVEREXPRESSESGENE,97772
GENEASSOCIATEDWITHCELLULARCOMPONENT,88880
GENECOVARIESWITHGENE,61606
CHEMICALBINDSGENE,25726


Node Properties

In [0]:
# 2.1 Node property keys
prop_keys = memgraph.execute_and_fetch(
    "MATCH (n) UNWIND keys(n) AS key RETURN DISTINCT key"
)
print("Node props:", [row["key"] for row in prop_keys])

Node props: ['nodeID', 'xrefCasRN', 'xrefDrugbank', 'commonName', 'sourceDatabase', 'typeOfGene', 'xrefEnsembl', 'xrefNcbiGene', 'geneSymbol', 'chromosome', 'xrefOMIM', 'xrefHGNC', 'xrefGeneOntology', 'pathwayId', 'pathwayName', 'xrefMeSH', 'xrefUberon', 'xrefNciThesaurus', 'xrefDiseaseOntology', 'xrefUmlsCUI', 'TF']


In [0]:
from collections import defaultdict
import pandas as pd

# first, get all labels
labels = [row["lab"] for row in memgraph.execute_and_fetch(
    "MATCH (n) UNWIND labels(n) AS lab RETURN DISTINCT lab"
)]

# then for each label, count how many nodes have each property
results = []
for lab in labels:
    # get all property keys present on any node of this label
    prop_keys = [r["key"] for r in memgraph.execute_and_fetch(f"""
        MATCH (n:{lab})
        UNWIND keys(n) AS key
        RETURN DISTINCT key
    """)]
    for key in prop_keys:
        cnt = list(memgraph.execute_and_fetch(f"""
            MATCH (n:{lab})
            WHERE n.{key} IS NOT NULL
            RETURN count(*) AS cnt
        """))[0]["cnt"]
        results.append({"label": lab, "property": key, "count": cnt})
        
df = pd.DataFrame(results)
# Assume `df` is your long form DataFrame from before
pivot_df = (
    df
    .pivot(index="property", columns="label", values="count")
    .fillna(0)
    .astype(int)
)

# Turn the index into a real column
pivot_df = pivot_df.reset_index()

# Now the "property" column is visible alongside all of the labels
display(pivot_df)

property,BiologicalProcess,BodyPart,CellularComponent,Disease,Drug,DrugClass,Gene,MolecularFunction,Pathway,Symptom,TranscriptionFactor
TF,0,0,0,0,0,0,0,0,0,0,519
chromosome,0,0,0,0,0,0,193279,0,0,0,0
commonName,12322,652,1695,34,16581,474,193279,3460,0,505,0
geneSymbol,0,0,0,0,0,0,193279,0,0,0,0
nodeID,12322,652,1695,34,16581,474,193279,3460,4516,505,519
pathwayId,0,0,0,0,0,0,0,0,4516,0,0
pathwayName,0,0,0,0,0,0,0,0,4516,0,0
sourceDatabase,12322,652,0,34,16581,474,193279,0,4516,505,519
typeOfGene,0,0,0,0,0,0,193279,0,0,0,0
xrefCasRN,0,0,0,0,16581,0,0,0,0,0,0


## Drug Gene and Drug Disease Interaction

In [0]:
import pandas as pd

# Aggregate outgoing edges from every Drug
query = """
MATCH (d:Drug)-[r]->(n)
RETURN type(r) AS rel, labels(n) AS target_labels, COUNT(*) AS cnt
ORDER BY cnt DESC
"""
rows = list(memgraph.execute_and_fetch(query))
df_out = pd.DataFrame(rows)
display(df_out)

rel,target_labels,cnt
CHEMICALBINDSGENE,List(Gene),25726
CHEMICALDECREASESEXPRESSION,List(Gene),21051
CHEMICALINCREASESEXPRESSION,List(Gene),18713
DRUGINCLASS,List(DrugClass),1945
DRUGTREATSDISEASE,List(Disease),9
DRUGCAUSESEFFECT,List(Disease),2


In [0]:
import pandas as pd

# 1) How many distinct Disease nodes do we have?
count_res = list(memgraph.execute_and_fetch("""
  MATCH (d:Disease)
  RETURN COUNT(d) AS num_diseases
"""))
num_diseases = count_res[0]["num_diseases"]
print(f"Total distinct diseases in the graph: {num_diseases}")

# 2) List all disease names
rows = list(memgraph.execute_and_fetch("""
  MATCH (d:Disease)
  RETURN DISTINCT d.commonName AS disease
  ORDER BY d.commonName
"""))
df = pd.DataFrame(rows)
print("Here are the first 20 diseases:")
display(df["disease"])

Total distinct diseases in the graph: 34
Here are the first 20 diseases:


0                                  ALZHEIMER DISEASE 10
1                                  ALZHEIMER DISEASE 18
2                                  ALZHEIMER DISEASE 19
3                                   ALZHEIMER DISEASE 2
4                                   ALZHEIMER DISEASE 4
5                                   ALZHEIMER DISEASE 5
6                       ALZHEIMER DISEASE 6, LATE-ONSET
7                ALZHEIMER DISEASE 9, SUSCEPTIBILITY TO
8                        ALZHEIMER DISEASE, FAMILIAL, 1
9     ALZHEIMER DISEASE, FAMILIAL, 3, WITH SPASTIC P...
10    ALZHEIMER DISEASE, FAMILIAL, 3, WITH UNUSUAL P...
11    ALZHEIMER DISEASE, FAMILIAL, WITH SPASTIC PARA...
12                 ALZHEIMER DISEASE, SUSCEPTIBILITY TO
13    ALZHEIMER DISEASE, SUSCEPTIBILITY TO, MITOCHON...
14                                 Alzheimer Disease 12
15                                 Alzheimer Disease 14
16                                  Alzheimer Disease 7
17                                  Alzheimer Di

In [0]:
import pandas as pd

# 1) Fetch all Disease nodes with their ontology cross‐refs
rows = list(memgraph.execute_and_fetch("""
  MATCH (d:Disease)
  WHERE d.xrefDiseaseOntology IS NOT NULL
  RETURN DISTINCT d.commonName   AS name,
                  d.xrefDiseaseOntology AS doid
  ORDER BY doid
"""))

df = pd.DataFrame(rows)
print(f"Total diseases with DOID: {df['doid'].nunique()} distinct IDs\n")

Total diseases with DOID: 19 distinct IDs



In [0]:
import pandas as pd

# 1) Raw Drug→Disease edges
rows = list(memgraph.execute_and_fetch("""
  MATCH (dr:Drug)-[r]->(di:Disease)
  RETURN dr.commonName   AS drug,
         type(r)         AS relation,
         di.commonName   AS disease
  ORDER BY disease, drug
"""))
df = pd.DataFrame(rows)

# 2) Count how many drugs per (disease, relation)
counts = (
    df
    .groupby(["disease","relation"])
    .size()
    .reset_index(name="drug_count")
)

# 3) Pivot to wide form
pivot = (
    counts
    .pivot(index="disease", columns="relation", values="drug_count")
    .fillna(0)
    .astype(int)
    .reset_index()
)

# 4) Build a list of all drugs per disease
drug_lists = (
    df
    .groupby("disease")["drug"]
    .apply(list)
    .reset_index(name="drugs")
)

# 5) Merge counts + lists
summary = pivot.merge(drug_lists, on="disease")

# 6) Display
display(summary)

disease,DRUGCAUSESEFFECT,DRUGTREATSDISEASE,drugs
Alzheimer's Disease,2,9,"List(Benzatropine, Donepezil, Galantamine, Haloperidol, Memantine, Naltrexone, Quetiapine, Rivastigmine, Ropinirole, Selegiline, Ziprasidone)"


## Two hop Drug - gene - Disease relation

In [0]:
import pandas as pd

# Aggregate outgoing edges from every Gene
query = """
MATCH (d:Gene)-[r]->(n)
RETURN type(r) AS rel, labels(n) AS target_labels, COUNT(*) AS cnt
ORDER BY cnt DESC
"""
rows = list(memgraph.execute_and_fetch(query))
df_out = pd.DataFrame(rows)
display(df_out)

rel,target_labels,cnt
GENEPARTICIPATESINBIOLOGICALPROCESS,List(BiologicalProcess),548285
GENEREGULATESGENE,List(Gene),263978
GENEINPATHWAY,List(Pathway),178991
GENEINTERACTSWITHGENE,List(Gene),147088
GENEHASMOLECULARFUNCTION,List(MolecularFunction),104752
GENEASSOCIATEDWITHCELLULARCOMPONENT,List(CellularComponent),88880
GENECOVARIESWITHGENE,List(Gene),61606
GENEASSOCIATESWITHDISEASE,List(Disease),508


In [0]:
import pandas as pd

# 1) Fetch every Drug–Gene–Disease two-hop
query = """
MATCH (d:Drug)-[:CHEMICALBINDSGENE|
               :CHEMICALINCREASESEXPRESSION|
               :CHEMICALDECREASESEXPRESSION]->(g:Gene)
MATCH (g)-[:GENEASSOCIATESWITHDISEASE]->(x:Disease)
RETURN d.commonName    AS drug,
       x.commonName    AS disease,
       COUNT(DISTINCT g) AS via_genes
ORDER BY via_genes DESC
"""
rows = list(memgraph.execute_and_fetch(query))

# 2) Load into a DataFrame
df = pd.DataFrame(rows)
print(f"Found {len(df)} distinct Drug→Disease candidates via genes\n")

Found 4352 distinct Drug→Disease candidates via genes



All common exploratory path between Drug and Disease

In [0]:
import pandas as pd
from functools import reduce

# Define each meta‐path by (query_fragment, column_name)
meta_paths = {
    "via_genes": (
        # 1) Drug→Gene via any of the three chemical relations
        """
        MATCH (dr:Drug)-[r]->(g:Gene)
        WHERE type(r) IN [
          'CHEMICALBINDSGENE',
          'CHEMICALINCREASESEXPRESSION',
          'CHEMICALDECREASESEXPRESSION'
        ]
        MATCH (g)-[:GENEASSOCIATESWITHDISEASE]->(dx:Disease)
        RETURN dr.commonName AS drug,
               dx.commonName AS disease,
               COUNT(DISTINCT g) AS via_genes
        """, "via_genes"
    ),
    "via_processes": (
        """
        MATCH (dr:Drug)-[r1:CHEMICALBINDSGENE]->(g:Gene)
        MATCH (g)-[:GENEPARTICIPATESINBIOLOGICALPROCESS]->(bp:BiologicalProcess)
        MATCH (bp)<-[:GENEPARTICIPATESINBIOLOGICALPROCESS]-(g2:Gene)
        MATCH (g2)-[:GENEASSOCIATESWITHDISEASE]->(dx:Disease)
        RETURN dr.commonName AS drug,
               dx.commonName AS disease,
               COUNT(DISTINCT bp) AS via_processes
        """, "via_processes"
    ),
    "via_pathways": (
        """
        MATCH (dr:Drug)-[r1:CHEMICALBINDSGENE]->(g:Gene)
        MATCH (g)-[:GENEINPATHWAY]->(pw:Pathway)
        MATCH (pw)<-[:GENEINPATHWAY]-(g2:Gene)
        MATCH (g2)-[:GENEASSOCIATESWITHDISEASE]->(dx:Disease)
        RETURN dr.commonName AS drug,
               dx.commonName AS disease,
               COUNT(DISTINCT pw) AS via_pathways
        """, "via_pathways"
    ),
    "via_molecular_func": (
        """
        MATCH (dr:Drug)-[r1:CHEMICALBINDSGENE]->(g:Gene)
        MATCH (g)-[:GENEHASMOLECULARFUNCTION]->(mf:MolecularFunction)
        MATCH (mf)<-[:GENEHASMOLECULARFUNCTION]-(g2:Gene)
        MATCH (g2)-[:GENEASSOCIATESWITHDISEASE]->(dx:Disease)
        RETURN dr.commonName AS drug,
               dx.commonName AS disease,
               COUNT(DISTINCT mf) AS via_molecular_func
        """, "via_molecular_func"
    ),
    "via_cellular_comp": (
        """
        MATCH (dr:Drug)-[r1:CHEMICALBINDSGENE]->(g:Gene)
        MATCH (g)-[:GENEASSOCIATEDWITHCELLULARCOMPONENT]->(cc:CellularComponent)
        MATCH (cc)<-[:GENEASSOCIATEDWITHCELLULARCOMPONENT]-(g2:Gene)
        MATCH (g2)-[:GENEASSOCIATESWITHDISEASE]->(dx:Disease)
        RETURN dr.commonName AS drug,
               dx.commonName AS disease,
               COUNT(DISTINCT cc) AS via_cellular_comp
        """, "via_cellular_comp"
    ),
    "via_TF": (
        """
        MATCH (dr:Drug)-[r1:CHEMICALBINDSGENE]->(tf:Gene)
        WHERE tf.TF = true
        MATCH (tf)-[:GENEREGULATESGENE]->(g:Gene)
        MATCH (g)-[:GENEASSOCIATESWITHDISEASE]->(dx:Disease)
        RETURN dr.commonName AS drug,
               dx.commonName AS disease,
               COUNT(DISTINCT tf) AS via_TF
        """, "via_TF"
    ),
    "via_drugclass": (
        """
        MATCH (dr:Drug)-[:DRUGINCLASS]->(dc:DrugClass)
        MATCH (dc)-[:DRUGINCLASS]->(d2:Drug)
        MATCH (d2)-[:CHEMICALBINDSGENE]->(g:Gene)
        MATCH (g)-[:GENEASSOCIATESWITHDISEASE]->(dx:Disease)
        RETURN dr.commonName AS drug,
               dx.commonName AS disease,
               COUNT(DISTINCT dc) AS via_drugclass
        """, "via_drugclass"
    ),
}

# 2) Run each query and collect into DataFrames
dfs = []
for cypher, col in meta_paths.values():
    rows = list(memgraph.execute_and_fetch(cypher))
    df = pd.DataFrame(rows)
    dfs.append(df)

In [0]:
for i, df in enumerate(dfs):
    print(f"DF {i} columns:", df.columns.tolist())

DF 0 columns: ['drug', 'disease', 'via_genes']
DF 1 columns: ['drug', 'disease', 'via_processes']
DF 2 columns: ['drug', 'disease', 'via_pathways']
DF 3 columns: ['drug', 'disease', 'via_molecular_func']
DF 4 columns: ['drug', 'disease', 'via_cellular_comp']
DF 5 columns: []
DF 6 columns: []


In [0]:
import pandas as pd
from functools import reduce

dfs = []
for name, (cypher, col) in meta_paths.items():
    rows = list(memgraph.execute_and_fetch(cypher))
    if rows:
        df = pd.DataFrame(rows)
    else:
        # create an empty DataFrame with the right schema
        df = pd.DataFrame(columns=["drug","disease", name])
    dfs.append(df)

In [0]:
df_all = reduce(
    lambda left, right: pd.merge(left, right, on=["drug","disease"], how="outer"),
    dfs
)

# fill NaNs in each path‐count column
for name in meta_paths.keys():
    df_all[name] = df_all[name].fillna(0).astype(int)

# optional any‐path sum
df_all["via_any"] = df_all[list(meta_paths.keys())].sum(axis=1)



In [0]:
display(df_all.head(2))

drug,disease,via_genes,via_processes,via_pathways,via_molecular_func,via_cellular_comp,via_TF,via_drugclass,via_any
Trametinib,Familial Alzheimer-like prion disease,1,16,4,0,3,0,0,24
Promazine,Familial Alzheimer-like prion disease,1,167,10,27,23,0,0,228


In [0]:
import pandas as pd

# 1) How many distinct Disease nodes do we have?
count_res = list(memgraph.execute_and_fetch("""
  MATCH (d:Drug)
  RETURN COUNT(d) AS num_drugs
"""))
num_drugs = count_res[0]["num_drugs"]
print(f"Total distinct drugs in the graph: {num_drugs}")

Total distinct drugs in the graph: 16581


In [0]:
# 1. Filter the high-strength pairs
df_hi = df_all[df_all["via_any"] > 100]

# 2. For each disease, sort its rows by via_any desc, then collect the drugs
grouped = (
    df_hi
    .sort_values(["disease","via_any"], ascending=[True, False])
    .groupby("disease") 
    .agg(drugs = ("drug", list), 
         strengths = ("via_any", list))
    .reset_index()
)

# 3. (Optional) Rename columns & inspect top 10 diseases by number of drugs
grouped["num_drugs"] = grouped["drugs"].str.len()
grouped = grouped.sort_values("num_drugs", ascending=False)

# 4. Show
display(grouped)

disease drugs strengths num_drugs Alzheimer Disease, Early Onset List(Fostamatinib, Fedratinib, Sunitinib, Nintedanib, Midostaurin, Zinc chloride, Bosutinib, Copper, Sorafenib, Dasatinib, Zinc acetate, Crizotinib, Zinc, Vandetanib, Imatinib, Erlotinib, Pazopanib, Ruxolitinib, Nilotinib, Neratinib, Tamoxifen, Axitinib, Gefitinib, Ceritinib, Clotrimazole, Ponatinib, Hexachlorophene, Infigratinib, Miconazole, Carvedilol, Amiodarone, Astemizole, Chlorpromazine, Raloxifene, Adenosine, Diethylstilbestrol, Niclosamide, Fluphenazine, Chloroquine, Cisplatin, Cannabidiol, Econazole, Artenimol, Quizartinib, Ibrutinib, Doxorubicin, Tofacitinib, Afatinib, Mesalazine, Nelfinavir, Paclitaxel, Regorafenib, Estradiol, Terfenadine, Clomifene, Acetylsalicylic acid, Clozapine, Celecoxib, Arsenic trioxide, Docetaxel, Bithionol, Haloperidol, Disulfiram, Thioridazine, Lapatinib, Troglitazone, Dopamine, Ibuprofen, Ebastine, Dexibuprofen, Estradiol benzoate, Tretinoin, Promethazine, Montelukast, Sulfasalazine, Thimerosal, Pseudoephedrine, Gentian violet cation, Epinephrine, Amitriptyline, Mitoxantrone, Zafirlukast, Trifluoperazine, Isotretinoin, Nifedipine, Prednisolone, Flunarizine, Loperamide, Caffeine, Tivozanib, Domperidone, Pimozide, Cabozantinib, Entrectinib, Vitamin E, Tacrolimus, Dobutamine, Brigatinib, Mifepristone, Ergotamine, Calcium phosphate dihydrate, Calcium Phosphate, Calcium citrate, Indomethacin, Theophylline, Donepezil, Valproic acid, Cyproheptadine, Dexamethasone, Encorafenib, Tannic acid, Imipramine, Prochlorperazine, Dacomitinib, Melatonin, Dabrafenib, Lenvatinib, Sulconazole, Apomorphine, Abemaciclib, Minocycline, Desipramine, Pralsetinib, Perphenazine, Aripiprazole, Danazol, Olanzapine, Zanubrutinib, Pentamidine, Risperidone, Acetylcysteine, Verapamil, Vemurafenib, Glycyrrhizic acid, Ketoconazole, Bortezomib, Adenosine phosphate, Nortriptyline, Paroxetine, Ziprasidone, Promazine, Diiodohydroxyquinoline, Naloxone, Clomipramine, Levodopa, Diclofenac, Pomalidomide, Doxepin, Lenalidomide, Fluoxetine, Fenofibrate, Quetiapine, Lamotrigine, Felodipine, Terazosin, Estradiol valerate, Estradiol dienanthate, Estradiol acetate, Estradiol cypionate, Asenapine, Sulindac, Trazodone, NADH, Cinnarizine, Perhexiline, Glyburide, Levothyroxine, Icosapent, Mianserin, Mezlocillin, Terconazole, Brexpiprazole, Ethinylestradiol, Pexidartinib, Rosiglitazone, Paliperidone, Budesonide, Progesterone, Curcumin, Cyclobenzaprine, Amoxapine, Benzethonium, Maprotiline, Sertindole, Acalabrutinib, Lovastatin, Methysergide, Clenbuterol, Zotepine, Cianidanol, Nicardipine, Ripretinib, Umbralisib, Copanlisib, Orphenadrine, Isoflurane, Thiothixene, Methylene blue, Ethanol, Erdafitinib, Propranolol, Nefazodone, Ketamine, Auranofin, Primaquine, Dronedarone, Cariprazine, Candesartan cilexetil, Flufenamic acid, Mirtazapine, Isoprenaline, Bromocriptine, Alitretinoin, Propafenone, Glutamic acid, Cabergoline, Thalidomide, Lauric acid, Lumateperone, Osimertinib, Menadione, Choline, Aripiprazole lauroxil, Oxiconazole, Vadadustat, Cyclosporine, Iloperidone, Masitinib, Fluspirilene, Meclofenamic acid, alpha-Linolenic acid, Sertraline, Betamethasone, Sirolimus, Lisuride, Buspirone, Tirbanibulin, Repotrectinib, Bepridil, Vorinostat, Doxazosin, Yohimbine, Testosterone, Tacrine, Methyldopa, Clemastine, Cisapride, Daunorubicin, Sildenafil, Lidocaine, Dihydroergotamine, Pimecrolimus, Metergoline, Venetoclax, Pergolide, Atorvastatin, Romidepsin, Metoclopramide, Idelalisib, Capsaicin, Prazosin, alpha-Tocopherol succinate, D-alpha-Tocopherol acetate, Ivermectin, Captopril, Cocaine, Telmisartan, Loxapine, Citric acid, Estrone, Dequalinium, Capivasertib, Liothyronine, Escitalopram, Acetaminophen, Cefapirin, Oxtriphylline, Vidarabine, Dihydroergocristine, Trimipramine, Itraconazole, Losartan, Formoterol, Methotrimeprazine, Leniolisib, Meperidine, Citalopram, Palbociclib, Lorlatinib, Mitotane, Amlodipine, Momelotinib, Oleic Acid, Methotrexate, Selumetinib, Fenfluramine, Quinidine, Tramadol,

# HeteroGraph

In [0]:
from torch_geometric.data import HeteroData
import pandas as pd
from collections import defaultdict

2025-05-30 08:15:39.266550: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748592939.290494    1249 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748592939.308830    1249 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-30 08:15:39.341623: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
